In [17]:
import numpy as np
import qiskit as qk
from qiskit import Aer
from math import ceil
from math import floor
%matplotlib inline

## Amplitude Encoding

In [18]:
def theta(data, i, j):
    n = int(np.log2(len(data)))
    
    idx1 = (2*j+1)*2**(n-i-1)
    idx2 = (j+1)*2**(n-i)
    idx3 = j*2**(n-i)
    idx4 = (j+1)*2**(n-i)
    
    if i==n-1:
        a1 = data[idx1]
    else:
        a1 = np.sqrt(np.sum(np.abs(data[idx1:idx2])**2))     
    
    a2 = np.sqrt(np.sum(np.abs(data[idx3:idx4])**2))
    if a2 == 0:
        return 0
    else:
        return 2*np.arcsin(a1/a2)

def interger_to_binary(integer, digits):
    binary = [int(b) for b in bin(integer)[2:]]
    binary = (digits-len(binary))*[0] + binary
    return binary
    
def amplitude_encoding(data, circuit, reg):
    N = data.shape[0]
    data = data.reshape(-1,)
    data = data/np.sqrt(np.sum(np.abs(data)**2))
    
    n = int(np.log2(N))
    c_reg = qk.ClassicalRegister(n, name="clas")
    reg = qk.QuantumRegister(n, name="reg")
    ancillae = qk.QuantumRegister(n, name="anc")
    circuit = qk.QuantumCircuit(c_reg, reg, ancillae)
    
    circuit.ry(theta(data, 0, 0), reg[0])
    for i in range(1, n):
        binary_ref = i*[0]
        circuit.x(reg[:i])
        for j in range(2**i):
            binary = interger_to_binary(j, i)
            
            for k, (b, b_ref) in enumerate(zip(binary, binary_ref)):
                if b != b_ref: circuit.x(reg[k]) 
            
            circuit.mcry(theta(data, i, j), reg[:i], reg[i], ancillae[:i])
            
            binary_ref = binary
    
    circuit.measure(reg, c_reg)
    
    return circuit

In [19]:
data = np.array([[1,2],[3,4], [5,6], [7,8]])
data = data.reshape(-1,)
data = data/np.sqrt(np.sum(np.abs(data)**2))

circuit = amplitude_encoding(data)
backend = Aer.get_backend('qasm_simulator')
#backend = Aer.get_backend('statevector_simulator')

job = qk.execute(circuit, backend, shots=1000)
result = job.result()
counts = result.get_counts(circuit)
counts = {key[::-1]:value for key, value in counts.items()}
print(data)
print(counts)

[0.070014   0.14002801 0.21004201 0.28005602 0.35007002 0.42008403
 0.49009803 0.56011203]
{'000': 7, '100': 114, '010': 40, '110': 256, '001': 17, '101': 175, '011': 82, '111': 309}


## Basis Encoding

In [12]:
def basis_encoding(x):
    M,N = x.shape
    
    clas_reg = qk.ClassicalRegister(N)
    loading_reg = qk.QuantumRegister(N, name = "loading")
    storage_reg = qk.QuantumRegister(N, name = "storage")
    ancillas = qk.QuantumRegister(N, name = "ancillas")
    branches = qk.QuantumRegister(2, name = "branches")
    
    
    circuit = qk.QuantumCircuit(clas_reg, loading_reg, storage_reg, ancillas, branches)
    circuit.x(branches[1])

    
    for i in range(M):
        for j in range(N):
            if x[i,j] == 1:
                circuit.x(loading_reg[j])
                circuit.cx(branches[1], storage_reg[j])
        
        circuit.cx(branches[1], branches[0])
        theta = -1/np.sqrt(M-i)
        circuit.cry(2*np.arcsin(theta), branches[0], branches[1])
        
        circuit.toffoli(loading_reg, storage_reg, ancillas)
        circuit.x(loading_reg)
        circuit.x(storage_reg)
        circuit.toffoli(loading_reg, storage_reg, ancillas)
        
        circuit.mcx(ancillas, branches[0])
        
        circuit.toffoli(loading_reg, storage_reg, ancillas)
        circuit.x(loading_reg)
        circuit.x(storage_reg)
        circuit.toffoli(loading_reg, storage_reg, ancillas)
        
        for j in range(N):
            if x[i,j] == 1:
                circuit.x(loading_reg[j])
                circuit.cx(branches[1], storage_reg[j])

    circuit.measure(storage_reg, clas_reg)
    return circuit

In [13]:
def float_to_binary(x, digits=4):
        binary_rep = []
        if x>0:
            binary_rep.append(0)
        else:
            binary_rep.append(1)
        
        x = abs(x)
            
        for i in range(digits):
            digit = floor(x*2**(i))         
            binary_rep.append(digit)
            x -= digit*2**(-i) 
        
        return binary_rep
    
def design_matrix_to_binary(X, digits=4):
    M,N = X.shape
    X_ = []
    
    for row in X:
        row_ = [float_to_binary(feature, digits) for feature in row]
        X_.append(row_)
    
    return np.array(X_).reshape(M, (digits+1)*N)

In [14]:
backend = Aer.get_backend('qasm_simulator')
#backend = Aer.get_backend('statevector_simulator')

M = 2
N = 2

x = np.random.normal(0, 1, size = (M,N))
print(x)
x = x/np.max(np.abs(x))
print(x)

x = design_matrix_to_binary(x,digits=3)
print(x)

circuit = basis_encoding(x)
job = qk.execute(circuit, backend, shots=1000)
result = job.result()
counts = result.get_counts(circuit)

counts = {key[::-1]:value for key, value in counts.items()}
print(counts)

[[-1.26257065  3.2480511 ]
 [-0.23797959 -1.118201  ]]
[[-0.38871638  1.        ]
 [-0.07326843 -0.34426829]]
[[1 0 0 1 0 1 0 0]
 [1 0 0 0 1 0 0 1]]
{'10010100': 517, '10001001': 483}


In [ ]:
reg1 = qk.QuantumRegister(2)
reg2 = qk.QuantumRegister(1)

circuit = qk.QuantumCircuit(reg1, reg2)
circuit.mcx(reg1, reg2)
circuit = circuit.decompose().decompose()
circuit.draw()
print(circuit.depth())

Rekkefølge på bits
kompleksitet til C^n x
dekomp av C^n ry
statevector av subsystem